<a href="https://colab.research.google.com/github/keithferns98/ML_Models/blob/main/HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Techniques Of Hyper Parameter 

Optimization

GridSearchCV

RandomizedSearchCV

Bayesian Optimization -Automate 

Hyperparameter Tuning (Hyperopt)

Sequential Model Based Optimization(Tuning 
a scikit-learn estimator with skopt)

Optuna- Automate Hyperparameter Tuning

Genetic Algorithms (TPOT Classifier)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [5]:
#df=df.dropna()

In [6]:
#df

In [7]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [8]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [9]:
X.shape,y.shape

((768, 8), (768,))

In [13]:

#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
y_pred=rf_classifier.predict(X_test)
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [14]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [15]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[89 18]
 [19 28]]
0.7597402597402597
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       107
           1       0.61      0.60      0.60        47

    accuracy                           0.76       154
   macro avg       0.72      0.71      0.72       154
weighted avg       0.76      0.76      0.76       154



In [17]:
#Manual Hyperparameter Tuning

model=RandomForestClassifier(n_estimators=300,criterion='entropy',max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
y_predmn=model.predict(X_test)
print(accuracy_score(y_test,y_predmn))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8246753246753247
[[89 18]
 [19 28]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       107
           1       0.61      0.60      0.60        47

    accuracy                           0.76       154
   macro avg       0.72      0.71      0.72       154
weighted avg       0.76      0.76      0.76       154



In [32]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
max_features=['auto','sqrt','log2']
max_depth=[int(x) for x in np.linspace(10,1000,10)]
min_samples_split=[1,2,3,4,5,7,9,10,14]
min_samples_leaf=[1,2,4,6,8,5,7,9,10,12]
random_grid={'n_estimators':n_estimators,
             'max_features':max_features,
             'max_depth':max_depth,
             'min_samples_split':min_samples_split,
             'min_samples_leaf':min_samples_leaf,
             'criterion':['gini','entropy']
             }
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 7, 9, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8, 5, 7, 9, 10, 12], 'criterion': ['gini', 'entropy']}


In [33]:

rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,
                               cv=3,verbose=2,random_state=100,n_jobs=-1)
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [34]:
rf_randomcv.best_params_ 

{'criterion': 'gini',
 'max_depth': 890,
 'max_features': 'log2',
 'min_samples_leaf': 8,
 'min_samples_split': 4,
 'n_estimators': 200}

In [35]:
best_random_grid=rf_randomcv.best_estimator_

In [36]:
y_pred_rf=best_random_grid.predict(X_test)
print(accuracy_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test,y_pred))

0.8311688311688312
[[98  9]
 [17 30]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       107
           1       0.61      0.60      0.60        47

    accuracy                           0.76       154
   macro avg       0.72      0.71      0.72       154
weighted avg       0.76      0.76      0.76       154



GridSearchCV

In [37]:
rf_randomcv.best_params_

{'criterion': 'gini',
 'max_depth': 890,
 'max_features': 'log2',
 'min_samples_leaf': 8,
 'min_samples_split': 4,
 'n_estimators': 200}

In [42]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'criterion':[rf_randomcv.best_params_['criterion']],
    'max_depth':[rf_randomcv.best_params_['max_depth']],
    'max_features':[rf_randomcv.best_params_['max_features']],
    'min_samples_leaf':[rf_randomcv.best_params_['min_samples_leaf'],
                       rf_randomcv.best_params_['min_samples_leaf']+2,
                       rf_randomcv.best_params_['min_samples_leaf']+4],
    'n_estimators':[rf_randomcv.best_params_['n_estimators']-150,rf_randomcv.best_params_['n_estimators']-100,
                    rf_randomcv.best_params_['n_estimators'],
                    rf_randomcv.best_params_['n_estimators']+150 , rf_randomcv.best_params_['n_estimators']+100],

    'min_samples_split':[rf_randomcv.best_params_['min_samples_split']-2,
                         rf_randomcv.best_params_['min_samples_split']-1,
                         rf_randomcv.best_params_['min_samples_split'],
                         rf_randomcv.best_params_['min_samples_split']+1,
                         rf_randomcv.best_params_['min_samples_split']+2]
             
}
print(param_grid)

{'criterion': ['gini'], 'max_depth': [890], 'max_features': ['log2'], 'min_samples_leaf': [8, 10, 12], 'n_estimators': [50, 100, 200, 350, 300], 'min_samples_split': [2, 3, 4, 5, 6]}


In [44]:
rf_grid=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  3.8min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [48]:
best_pred=grid_search.best_estimator_

In [46]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 890,
 'max_features': 'log2',
 'min_samples_leaf': 12,
 'min_samples_split': 2,
 'n_estimators': 100}

In [49]:
y_pred_grid=best_pred.predict(X_test)
print(accuracy_score(y_test,y_pred_grid))
print(confusion_matrix(y_test,y_pred_grid))
print(classification_report(y_test,y_pred_grid))

0.8116883116883117
[[97 10]
 [19 28]]
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       107
           1       0.74      0.60      0.66        47

    accuracy                           0.81       154
   macro avg       0.79      0.75      0.76       154
weighted avg       0.81      0.81      0.81       154



Automated Hyperparamter Tuning

1. Bayesian Optimization